In [ ]:
import tensorflow as tf
import numpy as np
import os, glob

In [ ]:
def lrelu(x, alpha=0.2):
    return tf.maximum(alpha*x, x)

def fc(x, n_units, dropout, activation=None):
    net = tf.layers.dense(x, n_units)
    net = tf.contrib.layers.layer_norm(net)
    if activation:
        net = activation(net)
    return tf.layers.dropout(net, dropout)

def conv1d(x, hidden_size, kernel_size, stride=1, dilation=1,
           pooling_size=0, dropout=0.0, activation=None):
    net = tf.layers.conv1d(x, hidden_size, kernel_size, stride, padding='same',
                           dilation_rate=dilation, activation=activation)
    if pooling_size:
        net = tf.layers.max_pooling1d(net, pooling_size, pooling_size, padding="same")
    return tf.layers.dropout(net, dropout)

def cnn(input_, n_classes, hp):
    net = input_
    for i in xrange(hp.n_conv_layers):
        net = conv1d(net, hp.hidden_size, hp.kernel_size, hp.stride, dilation=1,
                     pooling_size=hp.pooling_sizes[i], dropout=hp.dropout,
                     activation=hp.activation)
    for i in xrange(hp.n_dconv_layers):
        dilation= 2**(i + 1)
        tmp = conv1d(net, hp.hidden_size, hp.kernel_size, hp.stride, dilation=1,
                     pooling_size=0, dropout=hp.dropout, activation=hp.activation)
        net = tf.concat([net, tmp], axis=2)
    net = tf.contrib.layers.flatten(net)
    net = fc(net, hp.hidden_size, hp.dropout, activation=hp.activation)
    return fc(net, n_classes, hp.dropout, activation=hp.output_activation)

def rnn(input_, n_classes, hp): # RNN
    cell = tf.contrib.rnn.BasicLSTMCell(hp.hidden_size, forget_bias=1.0)
    
    values = tf.unstack(input_, axis=1)
    outputs, states = tf.contrib.rnn.static_rnn(cell, values, dtype=tf.float32)
    return fc(outputs[-1], n_classes, hp.dropout, activation=hp.output_activation)

def srnn(input_, n_classes, hp): # Stacked RNN
    cells = [tf.contrib.rnn.BasicLSTMCell(hp.hidden_size, forget_bias=1.0)
             for _ in xrange(hp.n_layers)]
    cell = tf.contrib.rnn.MultiRNNCell(cells)
    
    values = tf.unstack(input_, axis=1)
    outputs, states = tf.contrib.rnn.static_rnn(cell, values, dtype=tf.float32)
    return fc(outputs[-1], n_classes, hp.dropout, activation=hp.output_activation)

def birnn(input_, n_classes, hp): # Bidirectional RNN
    fw = tf.contrib.rnn.BasicLSTMCell(hp.hidden_size, forget_bias=1.0)
    bw = tf.contrib.rnn.BasicLSTMCell(hp.hidden_size, forget_bias=1.0)
    
    values = tf.unstack(input_, axis=1)
    outputs, fw_states, bw_states = tf.contrib.rnn.static_bidirectional_rnn(
        fw, bw, values, dtype=tf.float32)
    return fc(outputs[-1], n_classes, hp.dropout, activation=hp.output_activation)

In [ ]:
def cnn_hp(**kwargs):
    hp = tf.contrib.training.HParams()
    hp.n_conv_layers = 4
    hp.n_dconv_layers = 4
    hp.hidden_size = 64
    hp.kernel_size = 8
    hp.pooling_sizes = [2, 2, 2, 4]
    hp.stride = 1
    hp.dropout = 0.9
    hp.activation = lrelu
    hp.output_activation = tf.sigmoid
    hp.__dict__.update(kwargs)
    return hp

def rnn_hp(**kwargs):
    hp = tf.contrib.training.HParams()
    hp.hidden_size = 64
    hp.n_layers = 2 # Stacked RNN
    hp.dropout = 0.9
    hp.output_activation = tf.sigmoid
    hp.__dict__.update(kwargs)
    return hp

In [ ]:
import h5py
from scipy.io import loadmat

tmp = h5py.File('deepsea_train/train.mat', 'r')
train_input = tmp['trainxdata']
train_target = tmp['traindata']

tmp = loadmat('deepsea_train/valid.mat')
valid_input = tmp['validxdata']
valid_target = tmp['validdata']

def test_and_valid_batches(batch_size, input_, target):
    while True:
        for i in xrange(int(input_.shape[0]/batch_size)):
            yield input_[i*batch_size:(i+1)*batch_size], target[i*batch_size:(i+1)*batch_size]

def train_batches(batch_size, input_, target):
    while True:
        for i in xrange(int(input_.shape[-1]/batch_size)):
            yield (input_[:,:,i*batch_size:(i+1)*batch_size].transpose([2, 0, 1]),
                   target[:,i*batch_size:(i+1)*batch_size].transpose([1, 0]))

In [ ]:
tf.reset_default_graph()

input_placeholder = tf.placeholder(dtype=tf.float32, shape=[None, 1000, 4])
target_placeholder = tf.placeholder(dtype=tf.float32, shape=[None, 919])

# CNN
# hp = cnn_hp(n_dconv_layers=0)
# logits = cnn(input_placeholder, 919, hp)

# Dilated CNN
# hp = cnn_hp()
# logits = cnn(input_placeholder, 919, hp)

# RNN
# hp = rnn_hp()
# logits = rnn(input_placeholder, 919, hp)

# Stacked RNN
hp = rnn_hp()
logits = srnn(input_placeholder, 919, hp)

# Bidirectional RNN
# hp = rnn_hp()
# logits = birnn(input_placeholder, 919, hp)

loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
    logits=logits,labels=target_placeholder))
optimizer = tf.train.AdamOptimizer(1e-3).minimize(loss)
init_op = tf.global_variables_initializer()

print("Graph compiled.")
n_parameters = np.sum([np.prod(v.shape) for v in tf.trainable_variables()])
print("Number of parameters: %i" % n_parameters)

import logz

experiment = 'srnn_0'
logdir = os.path.join("/tmp/", experiment)
save_path = os.path.join(logdir, "model.ckpt")

saver = tf.train.Saver()
logz.configure_output_dir(logdir)
log_freq = 5
save_freq = 100

iterations = 10000
batch_size = 32

with tf.Session() as sess:
    if glob.glob(save_path + '*'):
        saver.restore(sess, save_path)
        print("Model restored.")
    else:
        sess.run(init_op)
        print("Model initialized.")
    
    batches = train_batches(batch_size, train_input, train_target)
    
    for i in xrange(iterations):
        input_, target = batches.next()
        _loss, _ = sess.run([loss, optimizer], feed_dict={
            input_placeholder: input_,
            target_placeholder: target,
        })
        if i % log_freq == 0:
            # print 'Iteration %d: loss = %.4f' % (i, _loss)
            logz.log_tabular('Iteration', i)
            logz.log_tabular('Loss', _loss)
            logz.dump_tabular()
        if i % save_freq == 0:
            save_path = saver.save(sess, save_path)
            print("Model saved in file: %s" % save_path)